# Table of content
- Task
- Draft
- Aciton
    - 0 读取数据文件并存储
    - 1 创建User类
    - 2 根据输入做出反应
    - 3 获得用户合法输入
    - 4 主函数
- Program code
- Difficulties
    - 0 提高文件读取性能（未完）
    - 1 类 or not
    - 2 如何更好实现用户指令查询功能
- Timelog

## Task
完成一个最简天气查询程序，运行在命令行界面，实现以下功能：

- 输入城市名，返回该城市的天气数据；
- 输入指令，打印帮助文档（一般使用 h 或 help）；
- 输入指令，退出程序的交互（一般使用 quit 或 exit）；
- 在退出程序之前，打印查询过的所有城市。
所用天气数据见 https://github.com/AIHackers/Py101-004/tree/master/Chap1/resource 中 weather_info.txt 文件。

## Draft

- 读取数据文件并存储
    - 读取
        - with open(path, 'r', encoding='utf-8') as f:
        - for line in f: f.read()
    - 存储
        - 使用 dict 存储（因字典取值时间复杂度是O(1)）
- 创建 User 类
    - 功能：记录用户合法输入，查询历史，输出历史
    - 属性
        - key(str): 合法用户输入
        - key_type:(str): 合法用户输入的类型, command/chinese
        - logs:(list):记录天气查询历史记录
    - 实例方法
        - add_log(self, search_result):将城市天气查询结果添加至 self.log 中
        - print_log(self):打印天气查询历史记录
- 根据输入作出反应
    - 指令
        - 封装功能至对应函数,参数统一为实例user，便于调用实例方法及实例属性
            - print_help(user)
            - print_history(user)
            - quit(user)
        - 保存指令名及对应函数变量至dict
            - command_dict = {
                'help': print_help,
                'history': print_history,
                'quit': quit,
                }
        - 判断是指令
            - if user.key_type == 'command':
        - 调用函数
            - response_func = command_dict[user.key]
            - response_func(user)
    - 中文(else)
        - 判断是中文
            - if user.key_type == 'chinese':
        - 作出响应
            - 查询天气 
                - result = weather_dict.get(user.key, None)
            - 保存查询记录
                - if result is not None: user.add_log(result)
- 获得用户合法输入
    - 获取输入
        - while-loop + try/except + input() 
    - 合法判断
        - 是否为指令 if iscommand(user_input): return user_input,'command'
        - 是否为中文 elif ischinese(user_input): return user_input, 'chinese'
        - 其他 else: continue

- 主体函数 main()
    - 读取数据
    - 初始化一个User实例 user
    - 获取用户输入
    - 根据输入调用不同函数

## Action
### 0 读取数据文件并存储

In [1]:
def data_from_file(path):
    """
    从本地 txt 文件中读取格式为"city, weather"的数据
    以 {city:weather} 这样的键值对保存至 dict 中，并返回

    :param path: (str), 含有天气数据的 txt 文件绝对/相对路径
    :return: (dict), key:value = city:weather 的 dict

    """
    
    dt = {}
    with open(path, 'r', encoding='utf-8') as f:
        for line in f:
            weather_ls = line[:-1].split(',')
            city = weather_ls[0]
            weather = weather_ls[1]
            dt[city] = weather
    return dt

### 1 创建User类

In [2]:
class User(object):
    """用于记录用户查询历史，输出历史"""

    def __init__(self):
        self.key = None
        self.key_type = None
        self.logs = []

    def add_log(self, search_result):
        """当用户使用城市名查询获得结果后，调用本方法添加至 log 属性中"""
        self.logs.append(search_result)

    def print_log(self):
        """打印logs内所有历史记录"""
        if len(self.logs) == 0:
            print('抱歉，今天您还没查过天气呢')
        else:
            for log in self.logs:
                print(log)

### 2 根据输入做出反应
#### 2.1 封装根据输入调用对应函数的函数
在写代码过程中:
- 在2.1部分构思好整体逻辑，把需要的函数以伪代码形式写下，思考其需要传入什么，返回什么
- 根据需求去2.2-2.3部分编写对应函数

In [4]:
def response_by_input(user, command_response_dt, weather_info_dt):
    """
    根据用户请求的关键词，调用对应函数
    :param user (instance): 用户实例
    :param command_response_dt (dict): 存放指令名称及对应功能函数变量的字典 
    :param weather_info_dt (dict): 存放城市天气信息的字典
    :return: 无返回
    """

    # 如果用户输入的请求是指令
    if user.key_type == 'command':
        response_func = command_response_dt[user.key]
        response_func(user)

    # 不是指令，那用户输入的请求就是中文
    else:
        weather_result = weather_search(user.key, weather_info_dt)
        # 如果查到了结果
        if weather_result is not None:
            print(weather_result)
            user.add_log(weather_result)

#### 2.2 封装指令函数

In [3]:
# 封装指令对应函数
def print_help(user):
    """打印帮助信息"""

    help_doc = """
    - 输入城市名，返回该城市的天气数据；
    - 输入指令 h 或 help，打印帮助文档；
    - 输入指令 quit 或 exit，退出本程序；
    - 输入指令 history，打印查询过的所有城市。
    """
    print(help_doc)


def print_history(user):
    """
    将存在于 log 中的历史查询记录打印出来
    """
    user.print_log()


def quit(user):
    """输出历史记录，然后退出程序"""
    from sys import exit

    user.print_log()
    exit(0)


# 保存指令名及对应函数变量至dict
command_function_dt = {
        'help': print_help,
        'history': print_history,
        'quit': quit,
    }

#### 2.3 封装天气查询函数

In [13]:
def weather_search(city, data_dt):
    """
    根据城市名查询对应的天气

    :param city: (str), 中文城市名
    :param data_dt: (dict), 以{city:weather} 格式存储城市天气信息的字典
    :return: 如果查到，返回 "city:weather" 格式的字符串;
              如果没查到，返回 None
    """
    weather = data_dt.get(city, None)

    if weather is None:
        print('抱歉，没有找到该城市，请确认城市名称输入正确')
        return None

    else:
        search_result = '{}:{}'.format(city, weather)
        return search_result

### 3 获得用户合法输入


In [6]:
def iscommand(user_input):
    """
    判断输入是否在已存指令中
    :param user_input (str): 用户输入
    :return: 如果输入在指令集中，True；反之，False
    """
    # 实际项目中，command_function_dt 放在 command_function.py 文件中
    # from command_function import command_function_dt
    if user_input in command_function_dt:
        return True
    else:
        return False


def ischinese(user_input):
    """
    检测字符是否只含有中文

    :param user_input:(str), 需要判断的字符串
    :return: 字符串全部为中文返回 True；否认返回False
    """

    for i in user_input:
        # 使用中文十六进制的 Unicode 字符集范围判断
        if i < '\u4e00' or i > '\u9fa5':
            return False

    return True

def get_user_input(prompt):
    """
    用于获得用户输入的合法命令或中文
    :param prompt (str): 提示语句
    :return: (user_input, 'command') 当user_input在是命令时；
              (user_input, 'chinese') 当user_input在是中文时；
    :raise: Exception 当用户输入导致异常，并退出程序
    """        

    while True:
        try:
            # 获得用户输入
            user_input = input(prompt)
        except Exception as err:
            from sys import exit
            print('你输入了什么？！..！@#￥%……&')
            print('Error:', err)
            exit(0)

        else:
            # 用户输入了命令
            if iscommand(user_input):
                return user_input, 'command'
            # 用户输入了中文
            elif ischinese(user_input):
                return user_input, 'chinese'
            # 非命令非中文
            else:
                print('输入非法')
                continue

### 4 主函数

In [14]:
def run(data_path):
    """
    主函数，调用后运行天气查询程序
    :param data_path (str): 存有天气数据文件的绝对或相对路径 
    :return: None
    """

    # 实际项目中，command_function_dt 放在 command_function.py 文件中
    # from command_function import command_function_dt

    # 读取天气数据
    weather_info_dt = data_from_file(data_path)

    # 初始化用户请求实例，需传入
    user = User()

    while True:
        # 获得用户输入的查询字符串
        prompt = '请输入城市名或指令,输入 help 获得帮助信息：  >'
        user_input = get_user_input(prompt)

        user.key = user_input[0]
        user.key_type = user_input[1]

        # 根据用户请求调用不同函数调用不同函数
        response_by_input(user, command_function_dt, weather_info_dt)


def main():
    data_path = '../resource/weather_info.txt'
    run(data_path)


if __name__ == '__main__':
    main()

请输入城市名或指令,输入 help 获得帮助信息：  >北京
北京:晴
请输入城市名或指令,输入 help 获得帮助信息：  >天津
天津:多云
请输入城市名或指令,输入 help 获得帮助信息：  >history
北京:晴
天津:多云
请输入城市名或指令,输入 help 获得帮助信息：  >help

    - 输入城市名，返回该城市的天气数据；
    - 输入指令 h 或 help，打印帮助文档；
    - 输入指令 quit 或 exit，退出本程序；
    - 输入指令 history，打印查询过的所有城市。
    
请输入城市名或指令,输入 help 获得帮助信息：  >quit
北京:晴
天津:多云


SystemExit: 0

C:\Anaconda2\envs\py36\lib\site-packages\IPython\core\interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## Code
### Version 1.0
代码放在4个文件,详见：[问天](https://github.com/thxiami/Py101-004/tree/master/Chap1/project)
各脚本文件内容：

- weather_forecast.py 
    - 主函数
- utils.py
    - 从天气信息文件获得数据的函数
    - 天气查询函数
    - 指令确认函数
    - 中文检查函数
    - 获得用户合法输入的函数
    - 对用户输入作出响应的函数
- user.py
    - User 类
- command_function.py
    - 指令功能函数
    - 指令字典: key(指令名称): value(函数变量)

## 可优化之处

### 0 提高文件读取性能
- 0 激发
    - 0.0 Scotting教练在[issue:ch1 任务难点「返回该城市天气数据」的思考](https://github.com/AIHackers/Py101-004/issues/42#issuecomment-322388219)下的怼

         > 楼上很多同学提到了性能问题，但目前我没有看到真正提升性能、节省内存的代码。给个提示，简单的办法是借助下标准库 csv 或 collections。

    - 0.1 由此引申4个问题(参考NBR-hugh同学笔记[WeatherInquiry_ExploringRecord.ipynb](https://github.com/NBR-hugh/Py101-004/blob/master/Chap1/note/CH1_WeatherInquiry_ExploringRecord.ipynb))
        - 如何判断程序的性能?
            - 有哪些指标?
                - 时间
                - 内存占用
            - 有哪些判断方法?
                - 时间：timeit 包
                - 内存：不知工具，可从资源管理器看
            - 有哪些优化方向?
                - 第三方包（底层使用c实现，可能更快）
                - 高级数据结构（collections）
        - csv 是什么? 适用场景? 如何使用?
        - collection 是什么? 适用场景?如何使用?
        - 为何它们可以提高性能？
- 1 探索    
    - 1.1 标准库 csv
        - 有人比较过 csv Python2，3不同打开方式和pandas中的csv_read的速度
    - 1.2 open()参数或功能选择
        - 读取方式
            - 按行读取
            - file.read(size) 逐步读取
            - file.read() 一次性读取
        - 文件打开模式
            - 'r'  Python3中打开时解码，返回 str 格式数据
            - 'rb' Python3中，返回 bytes 格式数据，可自行解码
            - 自己比较（打开1000次总用时）
                - 测试结果
                    - open('r'):0.36s
                    - open('rb'):0.14s
                - 总结
                    - 'rb'打开文件的确更快，猜测与'r'打开时解码有关
                    - 下一步验证'rb'解码后是否仍更快
                    
            - 发现有人比较过 [Reading ASCII file in Python3.5 is 2-3x faster as bytes than string](http://www.dalkescientific.com/writings/diary/archive/2016/08/03/bytes_and_unicode_read_performance.html)
        - 解码时机
            - 打开文件时解码
            - 打开文件后解码
        - 使用缓冲区
            - 搜索
            - 概念
            - 使用
            - 比选方案
                - buffering=0, 1, 1000, 10000, 50000
    - 1.3 codecs.open() 与 open() （比较打开1000次总用时）
        - 搜索
            -[io.open vs. codecs.open](https://mail.python.org/pipermail/python-list/2015-March/687124.html)，In Python 2, built-in open doesn't take an encoding argument,so if you want to use something other than binary mode or the default encoding, you were supposed to use codecs.open. 它可能是Python2的遗留产物，Python3 中open()可以带 encoding='' 参数，替代了它
        - 概念
        - 使用
            - with codecs.open(path, 'r', encoding='utf-8') as f:
                f.read()
            - with codecs.open(path, 'rb') as f:
                f.read()
        - 比选方案
            - open('r')
            - open('rb')
            - codecs.open('r')
            - codecs.open('rb')
        - 测试结果
            - open('r'):0.36s
            - open('rb'):0.14s
            - codecs.open('r'): 0.29 s
            - codecs.open('rb'):0.14 s
        - 总结
            - codecs.open('r') 比 open('r') 要快
            - codecs.open('rb') 与 open('rb') 同样速度
    - 1.4 标准库 json
        - 搜索
        - 概念
        - 使用
        - 比选方案(调用 60000 次函数，读取天气数据文件)
            - 方案1：open(path, 'r', encoding='utf-8')+ json.loads(data)
            - 方案2：open(path, 'rb') + json.loads(data, encoding='utf-8')
        - 结果
            - 'rb': 104.9 s 
            - 'r'： 116.8 s  
    - 1.5 使用 deque

### 1 类 or not

- **0 激发**
    - 学友 NBR-hugh 在[issue: 如何更好实现用户指令查询功能?](https://github.com/AIHackers/Py101-004/issues/64#issuecomment-323538602)提到：
    > 遵循一个原则: 如无必要,勿增实体.
    
    > 就 ch1 任务而言,函数封装就足够了,不必用到 类
        - 为什么要用类?用它使得程序更清晰了?还是一定用到他的什么特性?
        - 好像并没有,反而类使得整个程序结构更复杂了
    - 在 [commit-log](https://github.com/thxiami/Py101-004/commit/bae7e515f0fedccf8eca5dbd566ced53028a6306#commitcomment-23748016) 中，大妈和大猫教练都对本次作业引入类提出自己看法：
        - Zoom.Quiet:
        > 是也乎 ╮(╯▽╰)╭
        有必要动用类嘛?

        - simpleowen:
        > 我自己了解到的情况是面向对象编程有些争议，我自己的编程岗位上用到了，但不是大规模那种
        ![img](https://user-images.githubusercontent.com/17614465/29491415-e9122646-858d-11e7-83aa-99c6de3b23ca.jpg)
        
- **1 方案**
    - wilslee教练在[issue: 如何更好实现用户指令查询功能?](https://github.com/AIHackers/Py101-004/issues/64#issuecomment-323594738)下的回答很好。
    > @NBR-hugh 对这个问题的思考也很深入，很切合工程上的思考。
    **如无必要,勿增实体**. 这一原则也是在工程上会遵守的原则。同时工程上也会猜测需求的可能变化，去为程序预留更多的拓展空间，这也是 @thxiami 这里提到这个问题的一个目的，所以这是一个值得探索的问题。

    > 前者在需求有需要拓展时，可能会有重构代码的成本；后者则可能前期需要花一定心思和花费一定的时间成本，导致的是后期需求拓展不会太费劲。这点在工程上会是常有的事情。如何抉择也是根据实际具体的需求以及对业务变化的猜测(这点很多时候是基于经验和对业务的熟悉)来判断。
    - 既然是一个有争议的东西，其必然是有利有弊，在之后：
        - 实现个人 MVP 时若无需要，可不必引入
        - 后期为增加需求，可考虑引入

### 2 如何更好实现用户指令查询功能
- **0 激发**
    - 看到同学多用if/elif/elif 判断用户输入，然后调用对应函数。觉得这种方法不擅长处理指令较多情况/指令功能代码需修改时
    - 由此引发的问题：
        - 遵循**如无必要,勿增实体**,是否有必要优化该部分功能代码？
            - 学友 NBR-hugh 在[issue: 如何更好实现用户指令查询功能?](https://github.com/AIHackers/Py101-004/issues/64#issuecomment-323538602)提出假想需求
            - wilslee 教练指出这是一个根据经验来抉择的事情，参见“类 or not”中的回答
        - 如何优化？
            - 提 issue 问各位同学和教练 [issue: 如何更好实现用户指令查询功能?](https://github.com/AIHackers/Py101-004/issues/64)

- **1 方案**
    - ** 方案1** 封装为User的实例方法
        - 源于wilslee 教练[issue: 如何更好实现用户指令查询功能?](https://github.com/AIHackers/Py101-004/issues/64#issuecomment-323522253)建议

        - 可以把指令函数封装在User类的实例方法中，通过hasattr, getattr 和 callable 方法，根据给出的command name 判断实例是否有对应实例方法，有的话就调用，没有就说明是查询天气

        - 隐藏难点：指令名和实例方法名要一致吗？
            - 必须要有一个同名的实例方法，多指令对应同一功能时，可采用下面方法扩展，见代码
        - **示意代码**见下方cell

    - ** 方案2** 借鉴高手思路
        - scottming 教练在[issue: 如何更好实现用户指令查询功能?](https://github.com/AIHackers/Py101-004/issues/64#issuecomment-323571707)分析一个PYCON2017的演讲
        > 这个演讲不错：https://us.pycon.org/2017/schedule/presentation/518/ 可看看找找感觉，感觉我又挖坑了，命令行还是挺多门道的，下周开始任务难起来了，有兴趣的同学建议课程结束后再深入探索吧。
        - 还未看

In [15]:
class User():
    def __init__(self):
        pass

    def help(self):
        doc = "Help doc"
        print(doc)
    
    def h(self):
        self.help()

user = User()
command_input = 'help'
command_func = getattr(user, command_input, None)
if command_func and callable(command_func):
    command_func()
else:
    ... # 调用查询天气的函数

Help doc


## Timelog
- 阅读NBR-Hugh 笔记，学习架构 20min
- 撰写 Draft 50min
- 撰写 Action
     - 框架 10min
     - 代码 50min
- 编写优化部分探索记录
    - 类 or not 40min 23:50-0:27
    - 如何更好实现用户指令查询功能 30min
    - 提高文件读取性能 70min 未完